<a href="https://colab.research.google.com/github/Jagdish05/Fake-News-Detection/blob/main/2%20Layer%20NN%20(75k%20data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1IZ46FJv_F1fT5bghALp2AgXAGFH8eca-"})   # replace the id with id of file you want to access
downloaded.GetContentFile('FinalTrainSet.csv')

In [ ]:
import pandas as pd
data = pd.read_csv('FinalTrainSet.csv')
del data['Unnamed: 0']

data['Article'] = data['Headline'] + ': ' + data['Body']
del data['Headline']
del data['Body']
del data['Headline Word Count']
del data['Body Word Count']

data['Stance'] = data['Stance'].replace({'agree': 1})
data['Stance'] = data['Stance'].replace({'discuss': 1})
data['Stance'] = data['Stance'].replace({'disagree': 0})
data['Stance'] = data['Stance'].replace({'unrelated': 0})

articles_train = data['Article'].values
y_train = data['Stance'].values

downloaded1 = drive.CreateFile({'id':"1-tW5pvGu2uYgufcC9Lj8-hn2qU7wyxJP"})   # replace the id with id of file you want to access
downloaded1.GetContentFile('FinalTestSet.csv')

import pandas as pd
test = pd.read_csv('FinalTrainSet.csv')
del test['Unnamed: 0']

test['Article'] = test['Headline'] + ': ' + test['Body']
del test['Headline']
del test['Body']
del test['Headline Word Count']
del test['Body Word Count']

test['Stance'] = test['Stance'].replace({'agree': 1})
test['Stance'] = test['Stance'].replace({'discuss': 1})
test['Stance'] = test['Stance'].replace({'disagree': 0})
test['Stance'] = test['Stance'].replace({'unrelated': 0})

articles_test = test['Article'].values
y_test = test['Stance'].values

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(articles_train)

X_train = vectorizer.transform(articles_train)
X_test = vectorizer.transform(articles_test)

In [ ]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                248440    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 248,451
Trainable params: 248,451
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=100, verbose=False, batch_size=10)

loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.8941
Testing Accuracy:  0.8606
